# Byte Management Classes and Functions 

1) fix arithmetic decoder -- now to halfway, problem is the window size (divide by 0 means ho = lo - 1)
2) I need to check that the code is actually widening the window (but it is ... I have seen it)
3) everything else appears good -- it can now be fully run to create one big file, then to decode that file (although in a few cases, the active code must be changed
4) The full file runs, but there appear to be capitalization exceptions added.

In [2]:
# libraries
import string
import re
import numpy as np
import math

In [4]:
# load the file, remove bits from the byte array
# byte_store = loaded encoded text
# byte = int from 0 to 255, which holds the current byte, with used bits set to 0
# jj = index of byte_store (our current byte)
# ii = current bit within our current byte, from 0 (all available) to 7 (only the last bit is available)
# ll = len(byte_store)
class byte_mng:
    def __init__(self):
        self.byte_store = b'' # here I am filling the bytes, should be integers from 0 to 255
        self.byte = 0         # active byte

        self.ii = 0           # bits already read, goes from 0 to 7
        self.jj = 0           # bytes already read
        self.ll = 0           # length of the bytearray

        self.iio = 0
        self.jjo = 0
        self.rem = self.ll
        
    # unlike the previous version, the file is read first, not last
    # I need to check that this does what I think it does
    def load_file(self, filename):
        with open(filename, "rb") as f:
            self.byte_store = f.read() # the file should be put into a byte stream
            
        self.byte = self.byte_store[0]
        self.ii = 0         
        self.jj = 0          
        self.ll = len(self.byte_store) 

        self.iio = 0
        self.jjo = 0
        self.byteo = self.byte
        self.rem = self.ll
        
    # Once I open a file, I can start reading it. I never alter the data, but I will not access what has already been read. 
    # The first function assumes a known number of bits. We remove them one-by-one or 2 by 2 if we don't know how many we need.
    # sets self.iio and self.jjo, so the start of reading can be reset
    # return: value (int), whether or not we reach or superceed the length of the file
    def read_bits(self, lngth):
        done = False
        self.iio = self.ii
        self.jjo = self.jj
        self.byteo = self.byte
        self.rem = self.ll - self.jj
        
        # error-checking
        if self.ll == 0:
            print('File not defined.')
            done = True

        if self.rem == 0:
            print('End of file previously reached.')
            done = True

        ret_arr = [self.byte] # start with our return here
        used_bits = 8 - self.ii # used for length > remaining bit size
        k = (lngth - used_bits) // 8 # used for length > remaining bit size + 8

        #print("part1: ", self.ii, ", ", self.byte, ", ", lngth)
        
        # don't remove the bytes which don't need to be removed
        if (self.ii < (8 - lngth)) & (done == False): 
            mask = int(255) - int((2**(8 - lngth - self.ii) - 1))
            ret_arr[0] = ret_arr[0] & mask
            self.ii = self.ii + lngth
            mask = int(2**(8-self.ii) - 1)
            self.byte = self.byte & mask
            done = True

        #print("part1: ", self.ii, ", ", self.byte, ", ", lngth, ", ", done)

        #print(self.ii, self.jj)

        # full bytes
        if (done == False):
            if k > 0:
                for i in range(1, k+1):
                    if self.jj + i < self.ll:
                        ret_arr.append(self.byte_store[self.jj+i])
                    else:
                        print('Reached end of array.')
                        done = True
                        break
            if k >= 0:
                self.jj += k+1
                if self.jj < self.ll:
                    self.byte = self.byte_store[self.jj]
                else:
                    self.byte = 0 # end of array
                self.ii = 0 # return to beginning
                used_bits = used_bits + 8 * k
                
        if (done == False) & (lngth == used_bits):
            done = True
        elif (done == False) & (self.jj >= self.ll):
            print('Reached end of file')
            done = True
        elif (done == False) & (used_bits > lngth):
            print(lngth, used_bits)
            done = True

        #print(self.ii, self.jj)

        # and for the last byte
        if done == False:
            ret_arr.append(self.byte)
            #remaining bits
            rem_bits = lngth - used_bits
            #print("part2: ", rem_bits, ", ", lngth, ", ", used_bits)
            i = rem_bits
            mask = int(255) - int((2**(8-i) - 1))
            #if i > 8: print(i)
            ret_arr[-1] = ret_arr[-1] & mask
            self.ii = i
            mask = int((2**(8-i) - 1))
            self.byte = self.byte & mask

        #print(self.ii, self.jj, ret_arr)

        return self.byte_to_int(ret_arr, self.iio, self.ii)

    # this combines all bits into an integer
    def byte_to_int(self, byte_arr, start, end):
        lgt = len(byte_arr)
        num_bits = (lgt-1)*8 - start + end
        if end == 0:
            num_bits += 8
        out = 0
        for i in range(num_bits):
            # we want to take of the bit at index start + i
            ind = start + i
            kk = ind // 8
            ind = ind % 8
            pwr1 = int(2**(7-ind))
            num = pwr1 & byte_arr[kk] 
            #if ind < 7: num = num >> (7-ind)
            pwr_full = int(2**(num_bits - i - 1))
            if num > 0: out += pwr_full
        return(out)

    # if I read the next byte as a test, and find it is for the next section instead
    def revert(self):
        self.ii = self.iio
        self.jj = self.jjo
        self.byte = self.byteo
        self.rem = self.ll - self.jj
        
    def to_end_of_byte(self):
        if self.ii > 0:
            self.jj = self.jj + 1
        self.ii = 0
        self.rem = self.ll - self.jj
        if self.rem > 0:
            self.byte = self.byte_store[self.jj]
        else:
            self.byte = 0

    # now for the code for ternary code
    def read_ternary(self):
        # 00, 01, 10, 11
        # leading will be 1 or 0

        # 1 for 2 and 0 for 1
        # this gets the leading digit
        ter = [self.read_bits(1)+1]

        # repeat for the rest of the digits
        # 01 = 0, 10 = 1, 11 = 2
        code = 0
        while code >= 0:
            code = self.read_bits(2) - 1
            if code >= 0:
                ter.append(code)

        #print(ter, code)

        # now to return the number
        num = 0
        for i in range(len(ter)):
            #print(num)
            num += ter[i] * (3**(len(ter)-1-i))
            
        return num

    def read_list(self):
        word_arr = []
        cont = True
        while cont:
            word = self.read_word()
            if word == '':
                cont = False
            else:
                word_arr.append(word)
        return word_arr

    def read_word(self):
        word = ''
        cont = True
        while cont:
            char = self.read_char()
            if char == '':
                cont = False
            else:
                word = word + char
        return word

    def read_char(self):
        #000xx = stop
        #0010x = stop
        #0011x = ab
        #01xxx = c-j
        #10xxx = k-r
        #11xxx = s-z
        
        code_a = ord('a') - int('110', 2)
        #num = ord(cha) - ord('a') + code_a # the order is unchanged, but the numbering starts earlier
        #ord(cha) = num + ord('a') - code_a
        
        f3b = self.read_bits(3)
        if f3b == 0:
            char = ''
        elif f3b == 1:
            b4 = self.read_bits(1)
            if b4 == 0:
                char = '' # I could add a separate behavor here, but right now I have not.
            else:
                b5 = self.read_bits(1)
                if b5 == 0:
                    char = 'a'
                else:
                    char = 'b'
        else:
            b45 = self.read_bits(2)
            char_num = b45 + f3b*4 + code_a
            char = chr(char_num)

        return char

In [6]:
# I currently need to load three separate binary files, but they will be unified
# The order is as follows: a) encoding info and counts, b) capitalization data, c) text
byte_count = byte_mng()
#byte_count.load_file("test_counts_2")
byte_count.load_file("full_text_encode_Gould")

In [57]:
#print(byte_count.byte)

In [8]:
low = []
upp = []
for i in range(26):
    low.append(string.ascii_lowercase[i])
    upp.append(string.ascii_uppercase[i])
num = []
for i in range(10):
    num.append(string.digits[i])
sim = []

# Count Array
New variant:

 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
 - size: (bit size -> 7, 6, 6, 6, {5}, 5, ..., 5, {4}, 4, 4, 4, 4, {2}, 2 or just all 7)
 - p2: {'ABC'} -- The first three characters of the document (index 0 - 60). 
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - x = 61
 - p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - y = 61
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive character spacings. Counts are only important here, as the old counts can be calculated from these counts and the first 3 characters.
 - z = 61, to indicate end of file 
 - w = (bin(max(count4)) - 2) = number of bits in max count - ternary encoding
 - count4

Indicies encoded 0-60 (straight) 61 indicates need to increment the first index only if not obvious.

In [59]:
#byte_count.ii = 0
#byte_count.jj = 0
#byte_count.byte = byte_count.byte_store[0]

In [10]:
# first = p1

def remove_sym_arr(bm):
    sym_space = []
    bit_len = int(7)
    rem_chars = int(128 - 26*2 - 10)
    while rem_chars > 0:
        sym_space.append(int(bm.read_bits(bit_len)))
        rem_chars = rem_chars - sym_space[-1]
        bit_len = len(bin(rem_chars - 1)) - 2
        # print(sym_space, rem_chars, bit_len)
        # 0 is possible as the first character, but not as a spacing
        # if there are no special chars, the last number will be >= 66; 
        # but 0 may be used instead as the end after this
        if sym_space[-1] == 0 and len(sym_space) > 1: 
            break
    
    sym_space.pop() # remove the last, unused space, which ends the array
    print(sym_space)

    #print(len(sym_space))
    x1=x2=x3=False
    for i in range(1,len(sym_space)):
        sym_space[i] = sym_space[i] + sym_space[i-1]
        if sym_space[i] >= 48 and x1 == False: # digits
            x1 = True
            sym_space[i] += 10
        if sym_space[i] >= 65 and x2 == False: # uppercase letters
            x2 = True
            sym_space[i] += 26
        if sym_space[i] >= 97 and x3 == False: # lowercase layers
            x3 = True
            sym_space[i] += 26
    # now sym_space gives the ascii code for all the special characters

    print(sym_space)

    sym = []
    for i in sym_space:
        sym.append(chr(i))

    return sym

In [12]:
sim = remove_sym_arr(byte_count)
n = len(sim) + 26 + 10
size = len(bin(n)) - 2

ch = []
for i in range(48):
    if chr(i) in sim:    
        ch.append(chr(i))
ch = ch + num
for i in range(58, 97):
    if chr(i) in sim:    
        ch.append(chr(i))
ch = ch + low
for i in range(123, 128):
    if chr(i) in sim:
        ch.append(chr(i))
print(size, n, len(ch), sim)

[10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1]
[10, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 58, 59, 60, 62, 63, 91, 93, 124, 125]
6 61 61 ['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '>', '?', '[', ']', '|', '}']


In [893]:
# ['\n', ' ', '!', '\"', '$', '%', '&', '\'', '(', ')', '*', '+', ',', '-', '.', '/' ]
# sim2 = [':', ';', '<', '>', '?', '[', ']']
# sim3 = ['|', '}']
# {10(7), 22(6), 1(6), 1(6), 2(5), 1(5), 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
# size_sim_arr = [7, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 2, 2]
#len([10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 0])

In [14]:
# p2: {'ABC'} -- The first three characters of the document (index 0 - 60). 
context = []
#size = 6
for i in range(3):
    context.append(byte_count.read_bits(size))
print(context)

[55, 41, 52]


In [896]:
# so far correct

In [16]:
print(byte_count.ii, byte_count.jj, byte_count.byte, size)

6 17 0 6


In [898]:
# save my position, to not rerun everything after an error
#byte_count.ii = 6
#byte_count.jj = 17
#byte_count.byte = 0
#size = 6

In [18]:
# p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} 
# two indicies, first of length 61, second of leng
# there should be 1427 numbers loaded, each with a length of 6 bits

p3_arr = ([], [])
notdone = True
i = 0 # i is increasing by one at the beginning -- this gives the correct results
j = 0
notFirst= False

while notdone:
    jo = j
    j = byte_count.read_bits(size)
    if j <= jo and notFirst:
        i = i + 1
        if i >= len(ch):
            print('i greater than len')
            byte_count.revert()
            notdone = False
    elif j >= len(ch):
        i = i + 1
        if i < len(ch):
            j = byte_count.read_bits(size)
        else:
            notdone = False
    if notdone:
        p3_arr[0].append(i)
        p3_arr[1].append(j)
    notFirst = True

In [20]:
print(len(p3_arr[0]))

1427


In [904]:
#print(set(p3_arr[0]))

In [905]:
#print(p3_arr[1][0:30])

In [22]:
print(byte_count.ii, byte_count.jj, byte_count.byte)

2 1090 2


In [912]:
# save my position, to not rerun everything after an error
#byte_count.ii = 2
#byte_count.jj = 1090
#byte_count.byte = 2
#size = 6

In [24]:
# p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
# y = 61
# target size -- 12699

p4_arr = ([], [])
notdone = True
i = 0
j = 0
notFirst= False

while notdone:
    jo = j
    j = byte_count.read_bits(size)
    if j <= jo and notFirst:
        i = i + 1
        if i >= len(p3_arr[0]):
            print('i greater than len')
            byte_count.revert()
            notdone = False
    elif j >= len(ch):
        i = i + 1
        if i < len(p3_arr[0]):
            j = byte_count.read_bits(size)
        else:
            notdone = False
    if notdone:
        p4_arr[0].append(i)
        p4_arr[1].append(j)
    notFirst= True

In [26]:
print(len(p4_arr[0]))

12699


In [28]:
print(len(set(p4_arr[0])))

1427


In [34]:
#print(p4_arr[1][0:30])

In [ ]:
# strangely correct. need to compare actual numbers
# I think it is because the first index is 0 for the first one, while non-zero for later ones
# now it works properly, but in theory I should correct this so the code could in theory work with other texts

In [30]:
print(byte_count.ii, byte_count.jj, byte_count.byte)

0 10727 8


In [ ]:
# save my position, to not rerun everything after an error
#byte_count.ii = 0
#byte_count.jj = 10727
#byte_count.byte = 8
#size = 6

In [32]:
#  p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive character spacings. Counts are only important here, as the old counts can be calculated from these counts and the first 3 characters.
# z = 61, to indicate end of file 
# target size: 59088

p5_arr = ([], [])
notdone = True
i = 0
j = 0
notFirst= False

while notdone:
    jo = j
    j = byte_count.read_bits(size)
    if j <= jo and notFirst:
        i = i + 1
        if i >= len(p4_arr[0]):
            print('i greater than len')
            byte_count.revert()
            notdone = False
    elif j >= len(ch):
        i = i + 1
        if i < len(p4_arr[0]):
            j = byte_count.read_bits(size)
        else:
            notdone = False
    if notdone:
        p5_arr[0].append(i)
        p5_arr[1].append(j)
    notFirst= True

In [34]:
print(len(p5_arr[0]))

59088


In [ ]:
#also correct

In [36]:
print(byte_count.ii, byte_count.jj, byte_count.byte)

2 56752 28


In [93]:
# save my position, to not rerun everything after an error
#byte_count.ii = 2
#byte_count.jj = 56752
#byte_count.byte = 28
#size = 6

In [38]:
w = byte_count.read_ternary()
print(w)

16


In [40]:
#p2:  3 18
#p3:  1429 1091
#p4:  12848 10728
#p5:  61366 56753
#w =  16
#p6:  59088 174930

#w is now correct
print(byte_count.ii, byte_count.jj, byte_count.byte)

1 56753 0


In [42]:
print(len(byte_count.byte_store))

785358


In [44]:
# w = (bin(max(count4)) - 2) = number of bits in max count - ternary encoding

count4 = [] #np.zeros(len(p5_arr[0]), 'int')

# count4
for i in range(len(p5_arr[0])):
    count4.append(byte_count.read_bits(w))

whitespace2 = byte_count.read_bits(3) # this exists only in the full file encod
byte_count.to_end_of_byte() # should be the end now, but needed if the files are joined

print(whitespace2)

0


In [46]:
print(byte_count.ii, byte_count.jj, byte_count.byte, byte_count.ll)

0 174930 84 785358


In [48]:
print(max(count4))

42912


In [ ]:
# ends at the correct time, has the correct max count

In [50]:
print(count4[0:30], count4[-30:])

[4, 1, 4, 1, 3, 4, 3, 8, 5, 6, 18, 3, 24, 17, 1, 7, 6, 8, 8, 2, 1, 8, 2, 2, 1, 32, 98, 71, 1, 1] [4, 8, 1, 1, 17, 1, 1, 2, 1, 5, 1, 4, 1, 1, 2, 1, 4, 7, 1, 110, 8, 2, 2, 6, 1, 1, 1, 2, 1, 1]


# Pandas

# Capitalization Array

In [213]:
# this is because the files are separate
#byte_count = byte_mng()
#byte_count.load_file("test_capital")

In [52]:
abr_arr = byte_count.read_list()
name_arr = byte_count.read_list()

exc_size = byte_count.read_ternary()
exc_arr = []
for i in range(exc_size):
    exc_arr.append(byte_count.read_ternary())
byte_count.to_end_of_byte() # should be the end now, but needed if the files are joined

In [54]:
# should be ['GBS','GMT', 'BBC', 'FTL', 'SCU', 'CIA','HUD','DCU','CEO','EMP','GED','GCSE','XCOM','WXYZ','BABUM']
print(abr_arr)

['emp', 'ged', 'scu', 'ceo', 'cia', 'ftl', 'bbc', 'gmt', 'dcu', 'hud', 'gbs', 'xcom', 'wxyz', 'babum', 'gcse']


In [56]:
print(len(name_arr)) #756 is the target

756


In [58]:
print(name_arr[0:20], name_arr[-10:])

['hey', 'controller', 'harp', 'clark', 'pffff', 'bonfire', 'omega', 'taiwanese', 'gardn', 'africa', 'mrs', 'unable', 'nn', 'gn', 'art', 'thanagar', 'gaea', 'selina', 'nooo', 'chinese'] ['masters', 'bah', 'rrrrr', 'whow', 'orleans', 'ooo', 'haha', 'manta', 'wow', 'megan']


In [60]:
print(len(exc_arr), max(exc_arr)) #17850 4359

17850 4359


In [62]:
print(exc_arr[0:20])

[1, 5, 5, 6, 3, 6, 3, 6, 13, 6, 9, 7, 8, 7, 13, 18, 1, 4, 1, 3]


In [225]:
# I need to also calculate indicies from the given spacings
#note that encoding starts at 1, not 0, so should be shifted by one
#for i in range(len(exc) - 1):
#    j = len(exc) - i - 1
#    exc[j] = exc[j] - exc[j-1] # will be 1 if the numbers are next to each other, but encodings start at 1
#exc[0] = exc[0] + 1 # to start at 1, not 0

In [64]:
exc_arr[0] = exc_arr[0] - 1 # subtract one from the first index

for i in range(1,len(exc_arr)):
    exc_arr[i] = exc_arr[i] + exc_arr[i-1] # add the previous to go from spacing to index

In [66]:
print(exc_arr[0:20])

[0, 5, 10, 16, 19, 25, 28, 34, 47, 53, 62, 69, 77, 84, 97, 115, 116, 120, 121, 124]


In [ ]:
# I think the error was in the loading here. I checked in the past, but perhaps there was an error along the way. ...
# I loaded the sorted array instead of the unsorted array ...
# Works now.

# Capitalization Test
I can just load the text, decapitalizize, and see if it returns the original text. This will deal will get around the need to fix the text encoding algorithm.

In [ ]:
# just like the encoder

In [ ]:
# success

# Text Decoding

In [795]:
# this is because the files are separate
#byte_count = byte_mng()
#byte_count.load_file("text_encode")

In [68]:
# this is where are decoded text will be stored
#context = [ch.index('w'), ch.index('i'), ch.index('t')]
text = []
for i in context:
    text.append(ch[i])

In [70]:
print(text)

['w', 'i', 't']


In [434]:
# ch                             -- (character indicies) -- like before
# count4 [len(p5_arr[0])]        -- (our counts)
# c3_non0[:3][:len(num3_non0)]   -- (to find the first index based on character indicies)
# p3_arr[2][len] -- p3[0] is 0 to 60, p3[1] is 0 to 60
# p4_arr[2][len] -- p4[0] is 0 to len(p3) -1, p4[1] is 0 to 60
# p5_arr[2][len] -- p5[0] is 0 to len(p4) -1, p5[1] is 0 to 60

# first 3 characters
# context
# push context each time (new = {0,1,2}) ... 

# counts = count4, context = 'XYZ', char = 'W'
# def find_count(counts, context, char):
#    i,j,k,m = find_indicies(context, char)
#    return counts(ii, jj)

# I don't pass the counts like needed, so I removed this requirement
# context = [x, y, z] char = w, where w,x,y,z are all indicies
# I am storing the context in the form of indicies, not characters
#def find_count(context, char, counts = count4, c2 = p3_arr, c3 = p4_arr, c4 = p5_arr):

# find_count not used here, new version in the class

In [ ]:
# testing for find_count

In [ ]:
# - find the probabilities and borders based on the counts (give counts, find left and right border)
# def find_probs(counts):
# again not used for decoding

In [ ]:
# testing for find_probs

In [ ]:
# bit management is here in the encoder, but I already have one

In [72]:
# - keep track of and update l, h, bits
# {l_old, h_old, bits_old -> a, b -> h-l+1, h-l+1 / 2^n -> a+ term, b+ term -> l, 
#  h -> bin l, bin h -> enc update -> new bin l, new bin h, bits_new -> new l, new h}
class arith_code_mng:
    def __init__(self, context, byte_m = byte_count):
        # window size
        self.N = 32
        self.win_size = 2**self.N

        # left and right borders
        # I know I don't need so many, but it should be fine right now
        self.l = 0
        self.h = self.win_size - 1
        self.bits = 0
        self.lo = 0
        self.ho = self.win_size - 1
        self.ln = 0
        self.hn = self.win_size - 1

        # context
        self.context = context
        self.char = 0

        # counts
        # this is currently global

        # encoding
        self.byte_encode = byte_m
        self.duration_static = 0
        self.static_count = 0
        self.end = 0
        self.window = 0
        self.contin = True

        self.find_last_byte()
        self.find_first_byte()

    def find_last_byte(self):
        self.byte_l = self.byte_encode.ll
        self.cur_by = self.byte_encode.jj
        self.cur_bi = self.byte_encode.ii
        self.bit_end = 7

        # remove 0 bytes
        byte = 0
        while (byte == 0) and (self.byte_l >= self.cur_by):
            self.byte_l = self.byte_l - 1
            byte = self.byte_encode.byte_store[self.byte_l]
            self.duration_static = self.duration_static + 8

        # now for 0 bits
        i = 0
        while i < 8:
            i = i + 1 
            byte_new = (byte >> i) << i
            if byte_new != byte:
                break

        self.bit_end = 8 - i
            
        # we need to fix duration_static now
        # this gives the minimum duration
        #if self.duration_static > 8:
        #    self.duration_static -= self.bit_end
        self.duration_static += i - 1
        self.duration_static -= whitespace2

        # the last bit is now guarenteed to be 1 (unless there are no more remaining 1s)
        # the following condition marks this situation
        if (self.cur_by > self.byte_l) or ((self.cur_by == self.byte_l) and (self.cur_bi > self.bit_end)):
            self.contin = False

    def read_bit(self):
        if (self.cur_by < self.byte_l) or ((self.cur_by == self.byte_l) and (self.cur_bi <= self.bit_end)):
            bit = self.byte_encode.read_bits(1)
            self.cur_by = self.byte_encode.jj
            self.cur_bi = self.byte_encode.ii                                                                
        else:
            bit = 0
            self.end = self.end + 1 # I don't think I need this, but ...
        return bit

    def find_first_byte(self):
        byte = 0
        for i in range(self.N):
            bit = self.read_bit()
            byte = byte << 1
            byte = byte | bit
        self.window = byte
        
    def cont(self):
        # the last non-zero byte has been removed
        # and we need the count of unchanged cycles, which I think I forgot to encode
        if ((self.window == 2**(self.N - 1)) or (self.window == 0)) and ((self.cur_by > self.byte_l) or ((self.cur_by == self.byte_l) and (self.cur_bi > self.bit_end))):
            if (self.static_count >= self.duration_static) or (self.window == 0):
                self.contin = False
                
        return self.contin

    def shift_byte(self, n=1):
        for i in range(n):
            self.window = (self.window << 1) % self.win_size
            self.window = self.window | self.read_bit()

    def shift_byte_bits(self, n=1):
        for i in range(n):
            fb = (self.window // 2**(self.N-1)) << (self.N-1)
            self.window = (self.window << 1) % 2**(self.N-1) # removes 2 bits from the start, clearing a spot at the end and at the beginning
            self.window = self.window | self.read_bit() # add a bit to the end
            self.window = self.window | fb # return the first bit to the beginning

    # ----- here we have initialized everything,
    #      can check if we need to keep going
    #      and can find our byte string to read -----
            
    def remove_character(self):
        # find the location in the probability space for our byte array window 
        hl1_N = float(self.ho-self.lo+1) / float(self.win_size)
        w_pl = (self.window - self.lo) / hl1_N
        #print('w_pl, lo, ho, win')
        #print(bin(math.ceil(w_pl)), bin(self.lo), bin(self.ho), bin(self.window))

        # find the probabilities of various characters
        ind1, ind2, char_arr = self.find_nonzero_chars(self.context)
        a, char, b = self.find_window_chars(w_pl, ind1, ind2, char_arr)

        # update l and h like before
        a_pl = math.ceil(hl1_N * a)
        b_pl = math.ceil(hl1_N * (b + 1)) - 1

        self.l = self.lo + a_pl
        self.h = self.lo + b_pl
        
        #print(format(self.l, '032b'),",", format(self.h, '032b'),",", format(self.window, '032b'))
        #print("new l, h: ", bin(self.h), bin(self.l))
        enc_n, bit_n = self.update_window() # shifts the window and records how many bits need to be removed from l and h
        self.new_lh(enc_n, bit_n) # removes bits from l and h
        #print("remove: ", bin(self.hn), bin(self.ln), enc_n, bit_n)

        # updates for the next run
        # there is definately an error here in the encoding ...
        self.char = char
        self.lo = self.ln
        self.ho = self.hn
        self.update_context() # so that the next context uses the current character and removes the oldest

        return ch[char] # this array should not be global ... 

    def find_nonzero_chars(self, context):
        ind1 = 0
        ind2 = 0
        char_arr = []

        i1 = p3_arr[0].index(context[0])
        ii = p3_arr[1].index(context[1], i1)
        if p3_arr[0][ii] != context[0]:
            print('find_index issue, p3, ', p3_arr[0][ii], ", ", jj)

        #c3 for jj
        j1 = p4_arr[0].index(ii)
        jj = p4_arr[1].index(context[2], j1)
        if p4_arr[0][jj] != ii:
            print('find_index issue, p4, ', p4_arr[0], ", ", ii)
    
        #c4 for kk
        ind1 = p5_arr[0].index(jj)
        if jj < len(p4_arr[0])-1:
            ind2 = p5_arr[0].index(jj+1, ind1) - 1
        else:
            ind2 = len(p5_arr[0]) - 1

        char_arr = p5_arr[1][ind1:ind2+1]

        return ind1, ind2, char_arr
        # for comparison:
        # return sum(counts[k1:kk]), counts[kk], sum(counts[k, kk:k2])

    #      a, char, b = self.find_windows_chars(w_pl, ind1, ind2, char_arr)
    def find_window_chars(self, w_pl, ind1, ind2, char_arr):
        # this gets the counts
        char_counts = count4[ind1:ind2+1]

        den = float(sum(char_counts))
        b_fl = float(0)
        a = lb = 0
        num = float(0)
        i = b = rb = -1
        while rb < math.ceil(w_pl):
            a = lb
            i += 1
            num = num + float(char_counts[i])
            b_fl = num / den
            rb = math.ceil(b_fl * self.win_size) - 1
            b = rb
            lb = rb + 1 # for the next cycle
        
        return a, char_arr[i], b


    #enc_n, bit_n = self.update_window() # shifts the window and records how many bits need to be removed from l and h
    #self.new_lh(enc_n, bit_n) # removes bits from l and h
    # This is from the previous
    # This loads the bits into the array and updates bits, but so far doesn't update l and h
    def update_window(self):
        l = self.l
        h = self.h

        nbitload = 0
        nbits = 0

        #print(self.h-self.l)
        # both l and h are N bits long
        x = self.win_size
        #print(l / x, h / x)

        lb = 0
        hb = 0
        
        for i in range(self.N):
            #print("bits:", i, lb,hb, l/x, h/x)
            x = x // 2
            lb = l // x
            l = l % x
            hb = h // x
            h = h % x
            #print("bits:", i, lb,hb, l/x, h/x, self.bits)
            if lb == hb:
                self.shift_byte() # we are no longer loading
                while self.bits > 0:
                    #self.byte_encode.load_bits(1,1) # they are removed earlier
                    #self.shift_byte_bits()
                    self.bits -= 1
                nbitload += 1
            else:
                #print(i)
                break

        # the final 0s in the file
        if nbitload == 0:
            self.duration_static += 1
        else:
            self.duration_static = 0

        # widening the window
        # print(l / x, h / x)
        for i in range(self.N - nbitload - 1):
            x = x // 2
            lb = l // x
            l = l % x
            hb = h // x
            h = h % x
            if lb == 1 and hb == 0:
                nbits += 1
                self.bits += 1
                self.shift_byte_bits()
            else:
                break

        #print(self.h-self.l, nbitload, nbits)
        return nbitload,nbits

    # again, I need to check this and check the logic
    # should be unchanged
    def new_lh(self, nbitload, nbits):
        # removing bits
        self.ln = (self.l << nbitload) % self.win_size
        self.hn = (self.h << nbitload) % self.win_size
        self.hn = self.hn | (2**nbitload - 1) # fill the end with 1s, not 0s

        # widening of the window
        if nbits > 0:
            l_fb = (self.ln // 2**(self.N-1)) << (self.N-1)
            h_fb = (self.hn // 2**(self.N-1)) << (self.N-1)
            self.ln = (self.ln % 2**(self.N-1-nbits)) << nbits
            self.hn = (self.hn % 2**(self.N-1-nbits)) << nbits
            self.hn = self.hn | (2**nbits - 1) # fill the end with 1s, not 0s
            # and return the first bit to the array
            self.ln = self.ln | l_fb
            self.hn = self.hn | h_fb

    # unchanged
    def update_context(self):
        # This is not the most effective version of fofo list structure, but
        # I have only 3 elements, and it is the easiest to work with. 
        # I could get only 2 updates having a number which record the start, but
        # then I need to tell everything to access the array based on this.
        self.context[0] = self.context[1]
        self.context[1] = self.context[2]
        self.context[2] = self.char

        #print(self.context)

    # I need error-checking here to prevent using any routines if this has already been applied
    #def finalize(self):
    #    self.byte_encode.load_bits(1,1) # this is always 1, because 00, 01, 11 are the only possibilities, and 00 and 11 will already be removed.
    #    if self.duration_static > 0:
    #        self.byte_encode.load_bits(0, self.duration_static) # how many final characters the fraction was unchanged
            
        # now to finalize the bytestream
    #    a = 8 - self.byte_encode.bit_count
        # the encoding is already correct unless a = 8
    #    if a == 8:
    #        a = 0
            # remove the last zero byte
    #        b = self.byte_encode.byte_store.pop()
    #    return a

In [74]:
#context_start = [ch.index(text[0]), ch.index(text[1]), ch.index(text[2])]

encoder_new = arith_code_mng(context)
cond = encoder_new.cont()

while cond:
#for i in range(1900):
    text.append(encoder_new.remove_character())
    cond = encoder_new.cont()

text_out_f = ''.join(text)

print(text_out_f[0:2000])

#whitespace2 = encoder_new.finalize()

# I also need to add a number somewhere to indicate how many 0s are added to the end due to not being a full byte, and the size of the capitalization encoding,
# since neither this nor the captialization encoding have a definite end (I can add an end to the capitalization only when I know the text size)

with this ring [s01e01 - s01e26]

information
[story only and latest thread]
second half of 2010
author: mr. zoat [sb / sv]
compiled by: twilight d. dragon [sb / sv]
content: 26 episodes / 590 snippets
length: 754.363 words
status: completed

started on: 14/08/2013
completed on: 07/04/2015
rewrite: [01/01/2016 - 09/04/2016]
compliled on: 25/01/2017

table of contents

01 & 02) dependency day
03) welcome to happy hour
04) prod zone
05) skived
06) exfiltrator
07) acceptance
08) initiative
09) clueless
10) tracers
11) exaltations
12) home font
13) beta male
14) revolution
15) inhumanity
16) contingency
17) reordered
18) transparency
19) displaced
20) half-hearted
21) mirage
22) minutes
23) insincerity
24) preparation
25) unusual suspicions
26) gude-willie waught

*episode 01 & 02:* dependency day

5th july

my bed feels soft. did i go to sleep on top of the duvet? stars? did i leave the blinds open? i try to turn to the clock, but there's just more stars? i wake up pretty much instantly a

In [76]:
print(len(text))
text_out_f = ''.join(text)
print(text_out_f[-300:])

2150000
nd in motion."
we swim through concrete, metal and rock in the direction of our room.
"for the dark things cannot stand the light-"

we come out in the corridor. we try changing back to our human body-

"nerd."

"you recognised it, jackass."

"focus, gentlemen, please."

-but the internal defences i


In [78]:
filename = 'MrZoat_WithThisRingS01_Gould.txt'
with open(filename) as f:
    text3 = f.read()

In [80]:
text_out_f == text3.lower()[0:2150000]

True

# Capitalization

In [ ]:
#Due to not getting the whole text, I will need to be careful of the exceptions.

In [110]:
text = list(text_out_f)
text_out_f == text3.lower()[0:2150000]

True

In [112]:
i = text_out_f[0:-1].find('.')
j=0
k=-1
while i >= 0:
    # j is set to i, the last found location of a punctuation mark
    j = i
    # find the next letter after the punctuation mark
    x = re.search('[A-Za-z]', text_out_f[j:-1])
    # end of text, no more letters
    if x == None:
        break
    # k is the position of the letter to capitalize
    k = x.start()+j
    # switch case of the letter
    if not(any(text_out_f[k-1] == item for item in num)):
        text[k] = text[k].capitalize()
    # find next instance of any type of punctuation or a new paragraph
    i2 = k+1+text_out_f[k+1:-1].find('\n\n')
    cont = True
    i1 = k+1
    while (cont):
        x = re.search('[.?!]', text_out_f[i1:-1])
        cont = False
        if x == None:
            #we have no more punctuation, only '\n\n' relevent (but will be -1)
            i1 = i2
        else:
            #set our index
            i1 = i1+x.start()
            # we need to now add an additional check for '.. '
            if (text_out_f[i1+1]=='.' and text_out_f[i1+2]==' '):
                i1 = i1 + 2
                #print(k, i1, text_out_f[i1-3:i1+2])
                cont = True                
    if i2 < k+1:
        #we have no more new paragraphs, only punctuation relevant
        #note that if x = None as well, this does nothing, and they both indicate that there are no more instances
        i2 = i1
    i = min(i1,i2)
    # no more punctuation and no more new paragraphs
    if i < k+1:
        break

In [113]:
# code is still taking a while but much faster
# titles I am skipping, so the next is for the letter I in isolation

for i in range(len(text)):
    if text_out_f[i] == 'I' or text_out_f[i] == 'i':
        if i == 0:
            if any(text_out_f[1] == item for item in sim):
                text[i] = text[i].capitalize()
        elif i == len(text) - 1:
            if any(text_out_f[i-1] == item for item in sim):
                text[i] = text[i].capitalize()
        else:
            if any(text_out_f[i-1] == item for item in sim) and any(text_out_f[i+1] == item for item in sim):
                text[i] = text[i].capitalize()

In [114]:
# now to flip for the abr list
# k, kk are the index of interest
for i in abr_arr:
    k = 0      # reset the index
    l = len(i)
    for j in range(text_out_f.lower().count(i.lower())):
        k = k + 1
        kk = text_out_f[k:].lower().find(i.lower()) + k
        k = kk # set the index
        # if both sides are symbols 
        #if j == 0:
        if any(text_out_f[k-1] == item for item in sim) and any(text_out_f[k+l] == item for item in sim):
            for ii in range(l):
                text[k+ii] = text[k+ii].capitalize()
            #print(i, text[k-1:k+l+1])

In [115]:
# now to flip for the name list
# k, kk are the index of interest
for i in name_arr:
    k = 0      # reset the index
    l = len(i)
    for j in range(text_out_f.lower().count(i.lower())):
        k = k + 1
        kk = text_out_f[k:].lower().find(i.lower()) + k
        k = kk # set the index
        # if both sides are symbols 
        if any(text_out_f[k-1] == item for item in sim) and any(text_out_f[k+l] == item for item in sim):
            text[k] = text[k].capitalize()
            #print(i, text[k-1:k+l+1])

In [125]:
np.nonzero(np.array(exc_arr) > 2150000)

(array([ 9160,  9161,  9162, ..., 17847, 17848, 17849], dtype=int64),)

In [129]:
exc_arr[9159]

2149973

In [131]:
for i in exc_arr[:9160]:
    #if i > len(text):
    #    break
    text[i] = text[i].swapcase()

In [135]:
text[-1] = text[-1].swapcase() # because it is an 'i'

In [137]:
text2 = ''.join(text)
print(text2[0:200])

With This Ring [S01E01 - S01E26]

Information
[Story Only and Latest Thread]
Second Half of 2010
Author: Mr. Zoat [SB / SV]
Compiled By: Twilight D. Dragon [SB / SV]
Content: 26 Episodes / 590 Snippet


In [94]:
filename = 'decode_Gould.txt'
with open(filename, "w") as f:
    f.write(text2)

In [ ]:
#filename = 'MrZoat_WithThisRingS01_Gould.txt'
#with open(filename) as f:
#    text3 = f.read()

In [189]:
# First error. I must have messed up my exception array, because a previous version worked properly.
print(text2[1423733:1423750])
print(text3[1423733:1423750])

"Haha Haha haaha!
"Haha haha haaha!


In [193]:
np.nonzero(np.array(exc_arr) > 1423730)

(array([ 5749,  5750,  5751, ..., 17847, 17848, 17849], dtype=int64),)

In [203]:
exc_arr[5748:5751] # an erroneous exception was added

[1423021, 1423739, 1425764]

In [139]:
len(text2)

2150000

In [161]:
text2[0:1423000] == text3[0:1423000]

True

In [245]:
i1 = 1425000
i2 = 1660000
#i1 = 1790000
#i2 = 2010000
#i1 = 2034000
#i2 = 2150000
text2[i1:i2] == text3[i1:i2]

True

In [ ]:
#There are a bunch of them. I need to figure out what I changed. It was working before.

# Tests for Debugging

In [ ]:
# both algorithms get the same results -- not a problem with count4, or the access to count4
# I could move the update to both of the real versions

2**32

In [ ]:
# def find_window_chars(w_pl, ind1, ind2, char_arr):
# I need a w_pl within the window given

In [ ]:
# anything within the window gives what I need
# one error fixed, onto the next part